In [61]:
import numpy as np
import nibabel as nib
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from skimage import measure
from scipy import ndimage
import argparse
import os
import math
import time
import sys
import gc
from pathlib import Path
import shutil
from typing import List, Tuple
import psutil
import warnings
import k3d
from scipy.ndimage import map_coordinates

In [13]:
image_path = r"C:\Users\acer\Desktop\Project_TMJOA\Data\output\training_dataset_OA\test\0\57-16393 L_adjustedBG.nii.gz"
nib_image = nib.load(image_path)

print(nib_image.affine)

[[ -0.125   -0.      -0.      32.9375]
 [ -0.       0.125    0.     -30.0625]
 [  0.       0.      -0.125   32.875 ]
 [  0.       0.       0.       1.    ]]


In [49]:
img_data = nib_image.get_fdata(dtype=np.float32, caching='unchanged')

print(img_data.max(), img_data.min())
print(img_data.shape, img_data.size)

plot = k3d.plot()
obj = k3d.voxels(img_data, compression_level=1)
plot += obj
plot.display()

1374.0 -250.0
(255, 255, 255) 16581375


c:\Users\acer\anaconda3\envs\2dmodelGPU\Lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "uint8". A coerced copy has been created.
  warnings.warn(


Output()

In [59]:
img_data_thresholded = np.where(img_data > -250, 1, 0)

img_data_thresholded = img_data_thresholded.astype(np.uint8)

#img_data_thresholded = np.rot90(img_data_thresholded, axes=(0, 2))

print(img_data_thresholded.max(), img_data_thresholded.min())

plot = k3d.plot()
obj = k3d.voxels(img_data_thresholded, compression_level=1)
plot += obj
plot.display()

1 0


Output()

In [60]:
verts, faces, normals, _ = measure.marching_cubes(
                img_data_thresholded, 
                level=0.5,
                step_size=1
            )

plot = k3d.plot()            
obj = k3d.mesh(verts, faces, side='double')
plot +=  obj

plot.display()

Output()

In [ ]:
distances = 5
n_verts = len(verts)
n_distances = len(distances)

for i, (vertex, normal) in enumerate(zip(verts, normals)):
    if i % 10000 == 0:
        print(f"Processing vertex {i}/{len(verts)}")

    for j, dist in enumerate(distances):
        # Calculate sampling points along normals
        sample_points = vertex + dist * normals
        
        # Convert to image coordinates (transpose for map_coordinates)
        coords = sample_points.T  # Shape: (3, N)
        
        
        # Sample the image at these coordinates
        sampled_data[:, i] = map_coordinates(
            img_data, 
            coords, 
            order=1,
            mode='constant',
            cval=0
        )
    
    colors[i]
    return sampled_data